Class: DSC 670<br>
Name: Eros Perez<br>
Week: 3<br>
Problem: 4<br>
Link to Invoice: https://content.bellevue.edu/cst/dsc/670/dsc-670-exercise-invoice.pdf

In [111]:
import fitz  # PyMuPDF
import requests
import openai
import json

### Bring in pdf and extract text
In the following block of code we extract the data from the pdf link and then create a string called pdf_text that holds all the information in a long string.

In [124]:
#In this part of the code were are adding the link where the pdf is located
pdf_url = "https://content.bellevue.edu/cst/dsc/670/dsc-670-exercise-invoice.pdf"
pdf_path = "invoice.pdf"

#Request fetches the file and writes the content disk
response = requests.get(pdf_url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

# Using PyMuPDF, the text is extracted, then I loop through each page of text and finally join them together. 
#However, we only have one page for this example but this would work if their were many"
doc = fitz.open(pdf_path)
pdf_text = "\n".join(page.get_text() for page in doc)

### Use OpenAI API to transform the pdf into a desireable JSON format

In this step we start by pulling my personal API key into the client. I then create a system prompt that guides the LLM on what role it should be which is a helpfulassistant that extracts structured data from invoices. Then I tell it the important fields it should identify. Next, we sent the instructions to the client with an additional temperature parameter. We choose a low score so that accuracy is high and it doesn't allucinate. Finally, I structure the data in a JSON format. 

In [126]:
# This snippet of code pulls my API key from my computer as it was stored in a .zshrc file
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# This prompt guides the LLM on what it is and also tells the LLM what it should do. 
system_prompt = (
    "You are a helpful assistant that extracts structured data from invoices. "
    "Convert the following invoice text into clean JSON format, identifying fields such as: "
    "Invoice Number, Date, Bill To, Description, Quantity, Unit Price," 
    "Items over this amount qualift for an additional discount,"
    "Total, Credit, Subtotal, Tax as a percentage,Discount, Additional Discount as a percentage, Balance Due."
)

# This sends the request to GPT and instructs on the model, role and temperature to use. 
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Change to "gpt-4" if you have access
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": pdf_text}
    ],
    temperature=0.2
)

# Extract the structured JSON data
structured_json = response.choices[0].message.content
print(structured_json)


{
    "Invoice Number": "1111",
    "Date": "6/28/2024",
    "Bill To": {
        "Name": "Natasha Jones",
        "Company": "Central Beauty",
        "Address": "123 Main St., Manhattan, NY 98765",
        "Phone": "(321) 555-1234"
    },
    "Items": [
        {
            "Quantity": 1,
            "Description": "Item Number 1",
            "Unit Price": 2.00,
            "Amount": 2.00
        },
        {
            "Quantity": 1,
            "Description": "Item Number 2",
            "Unit Price": 2.00,
            "Amount": 2.00
        },
        {
            "Quantity": 1,
            "Description": "Item Number 3",
            "Unit Price": 2.00,
            "Amount": 2.00
        }
    ],
    "Items over this amount qualify for an additional discount": "$100",
    "Subtotal": 6.00,
    "Credit": 1000.00,
    "Tax": "9.80%",
    "Discount": "10%",
    "Additional Discount": "12%",
    "Balance Due": -994.20
}


### Evaluation

I am very impressed with the output of the LLM. It accurately extracted each portion of the pdf file that it was asked to do. I did notice that I had to correctly label certain parts such as Discount versus Additional Discount to accuratly extract both of them. I first tested with just Discount and it was attributed to 12%. Finally, when I correctly labeled both, both were accurately brought in. It also accurately added a percentage but only when I asked it too otherwise the percentage was displayed as a number which may cause problems in a pipeline. My final thoughts on this is that the model needs some tweaking to get it right because it does almost exactly what you ask it to do. If you are not precise then it will make its own judgement and may not accurately do what you asked such as turning  adiscount into a percentage unless explcitiely asked. 